# PyVista - NASA P3 Case Directory Read & Analysis

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import pyvista as pv

## Define Helper Functions to Slice the Internal Mesh

In [2]:
def slice_z_center(mesh):
    """Slice mesh through center in z normal direction, move to z=0."""
    slice_mesh = mesh.slice(normal='z')
    slice_mesh.translate((0, 0, -slice_mesh.center[-1]), inplace=True)
    return slice_mesh

## Define the Case Directory

In [3]:
case_directory = '/Users/jrobrien/Dissertation/data/solutions/NASA_noPylons_v2_tas120_aoa0_900T33/'

In [4]:
reader = pv.POpenFOAMReader(case_directory + 'NASA_noPylons_v2_tas120_aoa0_900T33.OpenFOAM')

## Subset the final run 

In [5]:
reader.time_values[-1]

786.0

In [6]:
%%time
reader.set_active_time_value(reader.time_values[-1])
reader.cell_to_point_creation = True
mesh = reader.read()

CPU times: user 4.59 s, sys: 641 ms, total: 5.23 s
Wall time: 5.21 s


In [7]:
internal_mesh = mesh["internalMesh"]
boundaries = mesh["boundary"]

In [8]:
internal_mesh

UnstructuredGrid (0x15cbe03a0)
  N Cells:    10691736
  N Points:   11526824
  X Bounds:   -2.000e+02, 2.000e+02
  Y Bounds:   -2.000e+02, 2.000e+02
  Z Bounds:   -2.000e+02, 2.000e+02
  N Arrays:   18

## Slice Mesh to View Aircraft - Plot

In [9]:
slice_internal_mesh = slice_z_center(internal_mesh)

2023-05-17 22:36:28.615 (   6.251s) [          1739F8]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.
2023-05-17 22:36:28.701 (   6.337s) [          1739F8]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.


In [10]:
slice_internal_mesh

PolyData (0x15cbe0ac0)
  N Cells:    56461
  N Points:   82061
  N Strips:   0
  X Bounds:   -2.000e+02, 2.000e+02
  Y Bounds:   -2.000e+02, 2.000e+02
  Z Bounds:   -6.805e-07, 6.805e-07
  N Arrays:   18

In [14]:
pt = pv.Plotter()
pt.add_mesh(slice_internal_mesh, scalars='U')
pt.show()

Widget(value="<iframe src='http://localhost:61764/index.html?ui=P_0x3ab026fa0_3&reconnect=auto' style='width: …